In [1]:
import pandas as pd 
import numpy as np
import regex as re
import seaborn as sns
from prepocessing import clean_text_df
from stempel import StempelStemmer
import regex as re
from joblib import load
from keras import models
import tensorflow as tf
from sklearn.manifold import TSNE
from models import data_vectorization

2023-08-09 09:39:19.634520: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 09:39:19.975200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 09:39:19.976681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 09:39:21.741138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('files/dataset_5_preprocessed.csv', index_col='id', parse_dates=['date'])

## Feature Engineering

## Cleaning text

In [24]:
stemmer = StempelStemmer.polimorf()
df = clean_text_df(df,stemmer)

Loading:   0%|          | 0/11368252 [00:00<?, ?bytes/s]

Loading: 100%|██████████| 11368252/11368252 [00:07<00:00, 1548424.08bytes/s]


In [3]:
path = 'models/'
vectorizer = load(path + 'tfidf.pkl')
selector = load(path + 'selector.pkl')
nn_c = models.load_model(path + 'nn_classifier.h5') 
enc = load(path + 'encoder.pkl')


In [4]:
nn_c.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                643200    
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 645825 (2.46 MB)
Trainable params: 64582

In [5]:
data = data_vectorization(df, vectorizer, selector, enc)
layers = nn_c.layers
new_model = tf.keras.models.Sequential(layers[:-1])
x_values, y_values = data
x_values.shape


KeyError: 'text_title'

In [28]:
new_model.input_shape

(None, 7546)

In [29]:
x_values = new_model(x_values)

2023-08-04 09:15:38.370059: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 132447392 exceeds 10% of free system memory.


In [30]:
tsne = TSNE(n_components=2,perplexity=50,n_iter=25000, verbose=1, random_state=123)
v = tsne.fit_transform(x_values) 

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 2194 samples in 0.001s...
[t-SNE] Computed neighbors for 2194 samples in 0.125s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2194
[t-SNE] Computed conditional probabilities for sample 2000 / 2194
[t-SNE] Computed conditional probabilities for sample 2194 / 2194
[t-SNE] Mean sigma: 0.013655
[t-SNE] KL divergence after 250 iterations with early exaggeration: 47.370224
[t-SNE] KL divergence after 5000 iterations: 0.131341


In [31]:
df_tsne = pd.DataFrame()
df_tsne["class"] = y_values
df_tsne["x"] = v[:,0]
df_tsne["y"] = v[:,1]
df_tsne["text"] = df["text"]
df_tsne["id"] = df.index
df_tsne.to_csv("tsne.csv")